<a href="https://colab.research.google.com/github/NancyYiWang/WildFireSmokePrediction/blob/main/GEOSVisualizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install netCDF4
!pip install matplotlib
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 107.4 MB/s eta 0:00:00


In [3]:
import os
from netCDF4 import Dataset

nc_directory = "/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R"

nc_files = [f for f in os.listdir(nc_directory) if f.endswith('.nc')]

nc_file_path = os.path.join(nc_directory, nc_files[0])
print(f"\nReading file: {nc_file_path}")

nc_data = Dataset(nc_file_path, 'r')

print(nc_data)


Reading file: /content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/ABI-L2-GFLSC-M6_v3r1_g16_s202401010001173_e202401010003546_c202401010005114.nc
<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    Metadata_Conventions: CF-1.6, Unidata Dataset Discovery v1.0
    standard_name_vocabulary: CF Standard Name Table v76
    institution: DOC/NOAA/NESDIS/NDE > S-NPP Data Exploitation, NESDIS, NOAA, U.S. Department of Commerce
    naming_authority: gov.noaa.nesdis.nde
    processing_level: NOAA Level 2
    production_site: NSOF
    production_environment: OE
    sensor_band_identifier: 2, 7, 14
    sensor_band_central_radiation_wavelength: 0.64um, 3.9um, 11.2um
    satellite_name: GOES-16
    instrument_name: GOES-R Series Advanced Baseline Imager
    project: Enterprise Processing System: GOES-R Fog/Low Stratus Cloud Product
    summary: Fog Depth and Flight Rule Probabilities
    history: Enterprise FLS Algorithm Version 2.3.1
    references:

In [4]:
print("\nVariables in the file:")
for var in nc_data.variables:
    dimensions = nc_data.variables[var].dimensions
    print(f"{var}: {dimensions}")

    if not dimensions:
            value = nc_data.variables[var][:]
            print(f"Value: {value}")


Variables in the file:
StartRow: ()
Value: 1
StartColumn: ()
Value: 1
Latitude: ('Rows', 'Columns')
Longitude: ('Rows', 'Columns')
Fog_Lrc_Mask: ('Rows', 'Columns')
Fog_Qf: ('Rows', 'Columns', 'QfConst')
Fog_Qpi: ('Rows', 'Columns', 'QPIConst')
MVFR_Fog_Prob: ('Rows', 'Columns')
IFR_Fog_Prob: ('Rows', 'Columns')
IFR_RHonly_Fog_Prob: ('Rows', 'Columns')
LIFR_Fog_Prob: ('Rows', 'Columns')
Fog_Depth: ('Rows', 'Columns')
Sfc_Temp_Bias: ('Rows', 'Columns')
Sfc_Temp_Bias_Global: ('Rows', 'Columns')
Refl_Chn2_StdDev: ('Rows', 'Columns')
Refl_Chn2_StdDev_Lrc: ('Rows', 'Columns')
Refl_Chn2_Stddev_Composite: ('Rows', 'Columns')
BT_Chn14_StdDev: ('Rows', 'Columns')
Emiss_Chn7_Atm_Correc: ('Rows', 'Columns')
Emiss_Chn7_Composite: ('Rows', 'Columns')
Refl_Chn7_Composite: ('Rows', 'Columns')
Sfc_Elevation: ('Rows', 'Columns')
Sfc_RH: ('Rows', 'Columns')
Sfc_RH_Global: ('Rows', 'Columns')
Max_RH_500ft_Lay: ('Rows', 'Columns')
Max_RH_1000ft_Lay: ('Rows', 'Columns')
Max_RH_3000ft_Lay: ('Rows', 'Column

In [7]:
!pip install earthengine-api xarray rasterio folium geopandas shapely -q

import ee
import os
import json
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import box
import folium
import gc
import traceback

ee.Authenticate()

nc_directory = "/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R"
geo_tiff_directory = "/content/drive/My Drive/WildFire/DATA/GeoTIFF"
geojson_directory = "/content/drive/My Drive/WildFire/DATA/GeoJSON"
os.makedirs(geo_tiff_directory, exist_ok=True)
os.makedirs(geojson_directory, exist_ok=True)

relevant_variables = [
    "Fog_Lrc_Mask",
    "MVFR_Fog_Prob",
    "IFR_Fog_Prob",
    "IFR_RHonly_Fog_Prob",
    "LIFR_Fog_Prob"
]

def convert_nc_to_geotiff(ds, variable, output_directory):
    try:
        data = ds[variable].values
        lon, lat = ds["Longitude"].values, ds["Latitude"].values

        lon = np.ma.filled(lon, np.nan) if np.ma.is_masked(lon) else lon
        lat = np.ma.filled(lat, np.nan) if np.ma.is_masked(lat) else lat

        if np.isnan(data).all() or data.shape != lon.shape or data.shape != lat.shape:
            print(f"Skipping variable '{variable}': Data is invalid or mismatched.")
            return None

        transform = from_origin(np.nanmin(lon), np.nanmax(lat), lon[0, 1] - lon[0, 0], lat[1, 0] - lat[0, 0])
        output_file = os.path.join(output_directory, f"{variable}.tif")
        with rasterio.open(output_file, "w", driver="GTiff", height=data.shape[0], width=data.shape[1],
                           count=1, dtype=data.dtype.name, crs="EPSG:4326", transform=transform) as dst:
            dst.write(data, 1)
        print(f"GeoTIFF saved at: {output_file}")
        return output_file
    except Exception as e:
        print(f"Error processing variable '{variable}': {e}")
        traceback.print_exc()
        return None

def convert_geotiff_to_geojson_manual(geo_tiff_path, output_directory):
    try:
        with rasterio.open(geo_tiff_path) as src:
            data = src.read(1)
            transform = src.transform

        mask = data != src.nodata
        shapes_gen = shapes(data, mask=mask, transform=transform)
        features = [
            {"type": "Feature", "geometry": geom, "properties": {"value": value}}
            for geom, value in shapes_gen if geom["type"] in ["Polygon", "MultiPolygon"]
        ]

        if not features:
            print("No valid features found.")
            return None

        geojson_data = {
            "type": "FeatureCollection",
            "features": features
        }

        output_file = os.path.join(output_directory, os.path.splitext(os.path.basename(geo_tiff_path))[0] + ".geojson")
        with open(output_file, "w") as f:
            json.dump(geojson_data, f)
        print(f"GeoJSON saved at: {output_file}")

        return output_file
    except Exception as e:
        print(f"Error converting GeoTIFF to GeoJSON: {e}")
        traceback.print_exc()
        return None

def visualize_layers_on_map(geojson_files, output_directory):

    manual_bounding_box = [-168.0, 24.0, -52.0, 72.0]

    for variable, geojson_file in geojson_files.items():
        map_center = [48.0, -100.0]
        map = folium.Map(location=map_center, zoom_start=4)

        map.fit_bounds([[manual_bounding_box[1], manual_bounding_box[0]], [manual_bounding_box[3], manual_bounding_box[2]]])

        try:
            folium.GeoJson(
                geojson_file,
                name=variable,
                style_function=lambda x: {"fillColor": "blue", "color": "red", "weight": 2, "fillOpacity": 0.5},
            ).add_to(map)
            map_path = os.path.join(output_directory, f"{variable}_map.html")
            map.save(map_path)
            print(f"Map for '{variable}' saved at: {map_path}")
        except Exception as e:
            print(f"Error visualizing variable '{variable}': {e}")

nc_files = [f for f in os.listdir(nc_directory) if f.endswith('.nc')]
if not nc_files:
    print("No NetCDF files found.")
else:
    nc_file_path = os.path.join(nc_directory, nc_files[0])
    print(f"Processing file: {nc_file_path}")

    try:
        ds = xr.open_dataset(nc_file_path, chunks={'Rows': 1000, 'Columns': 1000})
        geojson_files = {}
        for variable in relevant_variables:
            if variable in ds.variables:
                geo_tiff_file = convert_nc_to_geotiff(ds, variable, geo_tiff_directory)
                if geo_tiff_file:
                    geojson_file = convert_geotiff_to_geojson_manual(geo_tiff_file, geojson_directory)
                    if geojson_file:
                        geojson_files[variable] = geojson_file
                gc.collect()

        if geojson_files:
            map_path = '/content/drive/My Drive/WildFire/'
            visualize_layers_on_map(geojson_files, map_path)
        else:
            print("No valid GeoTIFF or GeoJSON files were created.")
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        traceback.print_exc()

Processing file: /content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/ABI-L2-GFLSC-M6_v3r1_g16_s202401010001173_e202401010003546_c202401010005114.nc
GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/Fog_Lrc_Mask.tif


<ipython-input-7-52ccf7c7acd5>:131: UserWarning: The specified chunks separate the stored chunks along dimension "Rows" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(nc_file_path, chunks={'Rows': 1000, 'Columns': 1000})
<ipython-input-7-52ccf7c7acd5>:131: UserWarning: The specified chunks separate the stored chunks along dimension "Columns" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(nc_file_path, chunks={'Rows': 1000, 'Columns': 1000})


Error converting GeoTIFF to GeoJSON: IllegalArgumentException: Points of LinearRing do not form a closed linestring


Traceback (most recent call last):
  File "<ipython-input-7-52ccf7c7acd5>", line 73, in convert_geotiff_to_geojson_manual
    features = [
  File "<ipython-input-7-52ccf7c7acd5>", line 75, in <listcomp>
    "geometry": mapping(shape(geom).simplify(0.01, preserve_topology=True)),
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/geo.py", line 101, in shape
    return Polygon(ob["coordinates"][0], ob["coordinates"][1:])
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/polygon.py", line 230, in __new__
    shell = LinearRing(shell)
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/polygon.py", line 104, in __new__
    geom = shapely.linearrings(coordinates)
  File "/usr/local/lib/python3.10/dist-packages/shapely/decorators.py", line 77, in wrapped
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/shapely/creation.py", line 171, in linearrings
    return lib.linearrings(coords, out=out, **kwargs)
shapely.errors.GE

GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/MVFR_Fog_Prob.tif
Error converting GeoTIFF to GeoJSON: IllegalArgumentException: Points of LinearRing do not form a closed linestring


...

GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/IFR_RHonly_Fog_Prob.tif
Error converting GeoTIFF to GeoJSON: IllegalArgumentException: Points of LinearRing do not form a closed linestring


Traceback (most recent call last):
  File "<ipython-input-7-52ccf7c7acd5>", line 73, in convert_geotiff_to_geojson_manual
    features = [
  File "<ipython-input-7-52ccf7c7acd5>", line 75, in <listcomp>
    "geometry": mapping(shape(geom).simplify(0.01, preserve_topology=True)),
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/geo.py", line 101, in shape
    return Polygon(ob["coordinates"][0], ob["coordinates"][1:])
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/polygon.py", line 230, in __new__
    shell = LinearRing(shell)
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/polygon.py", line 104, in __new__
    geom = shapely.linearrings(coordinates)
  File "/usr/local/lib/python3.10/dist-packages/shapely/decorators.py", line 77, in wrapped
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/shapely/creation.py", line 171, in linearrings
    return lib.linearrings(coords, out=out, **kwargs)
shapely.errors.GE

GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/LIFR_Fog_Prob.tif
Error converting GeoTIFF to GeoJSON: IllegalArgumentException: Points of LinearRing do not form a closed linestring


Traceback (most recent call last):
  File "<ipython-input-7-52ccf7c7acd5>", line 73, in convert_geotiff_to_geojson_manual
    features = [
  File "<ipython-input-7-52ccf7c7acd5>", line 75, in <listcomp>
    "geometry": mapping(shape(geom).simplify(0.01, preserve_topology=True)),
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/geo.py", line 101, in shape
    return Polygon(ob["coordinates"][0], ob["coordinates"][1:])
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/polygon.py", line 230, in __new__
    shell = LinearRing(shell)
  File "/usr/local/lib/python3.10/dist-packages/shapely/geometry/polygon.py", line 104, in __new__
    geom = shapely.linearrings(coordinates)
  File "/usr/local/lib/python3.10/dist-packages/shapely/decorators.py", line 77, in wrapped
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/shapely/creation.py", line 171, in linearrings
    return lib.linearrings(coords, out=out, **kwargs)
shapely.errors.GE

No valid GeoTIFF or GeoJSON files were created.
